In [10]:
%%writefile f2.template
<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8">

    <title>Collapsible Tree Example</title>

    <style>

 .node circle {
   fill: #fff;
   stroke: steelblue;
   stroke-width: 3px;
 }

 .node text { font: 12px sans-serif; }

 .link {
   fill: none;
   stroke: #ccc;
   stroke-width: 2px;
 }
 
    </style>

  </head>

  <body>

<!-- load the d3.js library --> 
<script src="http://d3js.org/d3.v3.min.js"></script>
 
<script>
var treeData = {myData};

// ************** Generate the tree diagram  *****************
var margin = {top: 20, right: 120, bottom: 20, left: 120},
 width = 1960 - margin.right - margin.left,
 height = 1500 - margin.top - margin.bottom;
 
var zoom = d3.behavior.zoom()
    .scaleExtent([1, 10])
    .on("zoom", zoomed);    
    
var i = 0;

var tree = d3.layout.tree()
 .size([height, width]);

var svg = d3.select("body").append("svg")
 .attr("width", width + margin.right + margin.left)
 .attr("height", height + margin.top + margin.bottom)
 .call(d3.behavior.zoom().on("zoom", function () {
    svg.attr("transform", "translate(" + d3.event.translate + ")" + " scale(" + d3.event.scale + ")")
  }))
 .append("g")
 .attr("transform", "translate(" + margin.left + "," + margin.top + ")");


root = treeData[0];
  
update(root);

function zoomed() {
  container.attr("transform", "translate(" + d3.event.translate + ")scale(" + d3.event.scale + ")");
}
   
function update(source) {

  // Compute the new tree layout.
  var nodes = tree.nodes(root).reverse(),
   links = tree.links(nodes);

  // Normalize for fixed-depth.
  nodes.forEach(function(d) { d.y = d.depth * 180; });

  // Declare the nodesâ€¦
  var node = svg.selectAll("g.node")
   .data(nodes, function(d) { return d.id || (d.id = ++i); });

  // Enter the nodes.
  var nodeEnter = node.enter().append("g")
   .attr("class", "node")
   .attr("transform", function(d) { 
    return "translate(" + d.y + "," + d.x + ")"; });

  nodeEnter.append("circle")
      .attr("r", 4.5)
      .style("fill", function(d) { return d._children ? "lightsteelblue" : "#fff"; });


  nodeEnter.append("text")
   .attr("dy", -15)
   .attr("text-anchor", "middle")
   .text(function(d) { return d.name; })
   .style("fill-opacity", 1);

  // Declare the linksâ€¦
  var link = svg.selectAll("path.link")
   .data(links, function(d) { return d.target.id; });

  // Enter the links.
  link.enter().insert("path", "g")
   .attr("class", "link")
   .attr("d", elbow);

function elbow(d, i) {
  return "M" + d.source.y + "," + d.source.x
	  + "H" + ((d.source.y+d.target.y) / 2)
      + "V" + d.target.x + "H" + d.target.y ;
}   
   
   
}

</script>
 
  </body>
</html>

Overwriting f2.template


In [11]:
North = ['Duke',"UNF",'San Diego St.','St. Johns','Utah','SF Austin','Georgetown','E. Wash.','SMU','UCLA',
         'Iowa St.','UAB','Iowa','Davidson','Gonzaga','N. Dak. St.']
South = ['Kentucky','New Hampshire','Cincinnati','Purdue','W. Virginia','Buffalo','Maryland','Valparaiso','Butler',
         'Texas','Notre Dame','Northeastern','Wichita St.','Indiana','Kansas','N. Mex. St.']
East = ['Villanova','Lafayette','NC State','LSU','N. Iowa','Wyoming','Louisville','UC Irvine','Providence','Boise St.',
        'Oklahoma','Albany','Michigan St.',"Georgia",'Virginia','Belmont']
West = ['Wisconsin','Coast. Car.','Oregon',"Oklahoma St.",'Arkansas','Wofford','N. Carolina','Harvard','Xavier','BYU',
        'Baylor','Georgia St.','VCU','Ohio St.','Arizona','Texas So.']

bracket2015 = np.append([],North)
bracket2015 = np.append(bracket2015,South)
bracket2015 = np.append(bracket2015,East)
bracket2015 = np.append(bracket2015,West)
bracket2015 = [bracket2015, np.append(East,South)]
bracket2015.append(East)
bracket2015.append(East[0:8])
#bracket2015 = []
bracket2015.append(East[0:4])
bracket2015.append(East[0:2])
bracket2015.append([East[0]])
print(np.shape(bracket2015)) 
print(bracket2015[6][0])

(7,)


In [13]:
import numpy as np

def treeMaker(size): 
    
    levels = np.log(size)/np.log(2)
    if np.log(size)%np.log(2) > 0:
        size = 2 ** int(levels)
    bracket = []     
    
#    bracket = [{"name":  "0" , "parent" : "0" , "children" : ""}]
#    for level in range(int(levels)):
#        bracket[0]['children'] = [{"name": level , "parent" : level , "children" : bracket},{"name": level ,
#                                    "parent" : level , "children" : bracket}]
    bracket = [{"name": team+1 , "parent" : "" , "children" : ""} for team in np.arange(size)]
#        bracket.append({"name": level , "parent" : level , "children" : bracket})
    bracket = level(bracket)    
        
    return bracket

def level(bracket):
    newbracket = []
    for loop in np.arange(0, np.size(bracket),2):
        newbracket.append( {"name": "winner" , "parent" : "" , "children" : [bracket[loop],bracket[loop+1]]})
    if np.size(newbracket)>1:
        newbracket = level(newbracket)
    return newbracket

def namedTree(bracket):
    newbracket = []
    for loop1 in bracket:
        level = []
        for loop2 in loop1:
            level.append( {"name": loop2 , "parent" : "" , "children" :""})
        newbracket.append(level)
    return newbracket

def combineData(arr):
    for loop1 in np.arange(1, np.size(arr)):
        for loop2 in np.arange(0, np.size(arr[loop1])):
            arr[loop1][loop2]['children'] = [arr[loop1-1][2*loop2], arr[loop1-1][2*loop2+1]]
            #print(arr[loop1][loop2])
    return arr[-1]

    

In [19]:
myTree = namedTree(bracket2015)
bracketology = combineData(myTree)

In [18]:
am = treeMaker(64)  

In [16]:
from IPython.display import IFrame
import re

def replace_all(txt,d):
    rep = dict((re.escape('{'+k+'}'), str(v)) for k, v in d.items())
    pattern = re.compile("|".join(rep.keys()))
    return pattern.sub(lambda m: rep[re.escape(m.group(0))], txt)    


count=0
def serve_html(s,w,h):
    import os
    global count
    count+=1
    fn= '__tmp'+str(os.getpid())+'_'+str(count)+'.html'
    with open(fn,'w') as f:
        f.write(s)
    return IFrame('files/'+fn,w,h)

def f2(w,h,data):
    d={'myData' : data}
    with open('f2.template','r') as f:
        s=f.read()
    s= replace_all(s,d)        
    return serve_html(s,w+30,h+30)

In [17]:
f2(900,500,bracketology)